In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,has_date,jieba_cut_content,content_sema
0,1.0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,8.456151
1,0.0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,7.486084
2,1.0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,7.175171
3,1.0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,7.565682
4,1.0,1,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,2.063409
5,1.0,1,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,7.143747
6,0.0,1,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,4.807568
7,1.0,1,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,6.593684
8,1.0,1,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,7.611074
9,0.0,1,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64619
Data columns (total 4 columns):
label                64284 non-null float64
has_date             64284 non-null int64
jieba_cut_content    64284 non-null object
content_sema         64284 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.5+ MB


In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_date,jieba_cut_content,content_sema
18810,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61632,1,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,1,http : / / tengyingge . blogchina . co...,8.289843
5881,1,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26338,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
vectorizer = CountVectorizer()
transfromer = TfidfTransformer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20)
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str'))
vectorizer_model = vectorizer.fit(jieba_cut_content)
df0 = vectorizer_model.transform(jieba_cut_content)
transfromer_model = transfromer.fit(df0)
df1 = transfromer_model.transform(df0)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)
data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220013 -0.237471 -0.039913 -0.070175  0.002541 -0.160743   
1  0.014968  0.064741  0.027001 -0.004667  0.008155  0.007437  0.016385   
2  0.011862  0.072033  0.047002  0.001427 -0.002089  0.035884  0.009183   
3  0.019714  0.117440  0.132444  0.060408 -0.103508 -0.074829 -0.012299   
4  0.346257  0.215338 -0.234024 -0.043091 -0.071088 -0.000006 -0.162576   

         7         8         9         10        11        12        13  \
0 -0.009818  0.202096 -0.008903 -0.199006  0.038951 -0.064485 -0.429894   
1  0.046021  0.029497 -0.000089  0.007913  0.023419  0.041893  0.012468   
2 -0.012264 -0.016601  0.000216  0.001207 -0.004016 -0.002082  0.002665   
3 -0.005582 -0.027207 -0.000218 -0.017020 -0.013031 -0.003020 -0.019128   
4 -0.009123  0.199409 -0.008701 -0.194635  0.037216 -0.060793 -0.418122   

         14        15        16        17        18        19  
0 -0.478702  0.208254  0.096033 -0

In [9]:
# 3.2 数据合并
data['has_date'] = list(x_train['has_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220013 -0.237471 -0.039913 -0.070175  0.002541 -0.160743   
1  0.014968  0.064741  0.027001 -0.004667  0.008155  0.007437  0.016385   
2  0.011862  0.072033  0.047002  0.001427 -0.002089  0.035884  0.009183   
3  0.019714  0.117440  0.132444  0.060408 -0.103508 -0.074829 -0.012299   
4  0.346257  0.215338 -0.234024 -0.043091 -0.071088 -0.000006 -0.162576   

          7         8         9      ...             12        13        14  \
0 -0.009818  0.202096 -0.008903      ...      -0.064485 -0.429894 -0.478702   
1  0.046021  0.029497 -0.000089      ...       0.041893  0.012468  0.012376   
2 -0.012264 -0.016601  0.000216      ...      -0.002082  0.002665 -0.004040   
3 -0.005582 -0.027207 -0.000218      ...      -0.003020 -0.019128  0.001858   
4 -0.009123  0.199409 -0.008701      ...      -0.060793 -0.418122 -0.468844   

         15        16        17        18        19  has_date  content_sem

In [10]:
%%time
knn = KNeighborsClassifier(n_neighbors=5)
model = knn.fit(data, y_train)

Wall time: 988 ms


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(vectorizer_model.transform(jieba_cut_content_test))))
data_test['has_date'] = list(x_test['has_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())

          0         1         2         3         4         5         6  \
0  0.537283  0.009051 -0.087601  0.028966  0.000860 -0.032655 -0.157805   
1  0.210902  0.131679 -0.090074 -0.086947 -0.033331  0.039711  0.006968   
2  0.045151  0.277282  0.233904  0.051026 -0.074015 -0.016335  0.010576   
3  0.032907  0.163067  0.113492  0.002454  0.053730  0.008262  0.004300   
4  0.035278  0.065208  0.026070 -0.023730  0.072928 -0.005847 -0.003259   

          7         8         9      ...             12        13        14  \
0  0.008966 -0.134587  0.009853      ...       0.081400 -0.006030 -0.017206   
1 -0.008254  0.011470 -0.000222      ...      -0.025285 -0.081762 -0.056922   
2 -0.014448  0.099936 -0.003569      ...      -0.066194  0.036820 -0.029243   
3 -0.020169  0.117693 -0.002549      ...      -0.061388  0.058911 -0.011851   
4 -0.008277  0.008778  0.000645      ...      -0.000417  0.003177 -0.006203   

         15        16        17        18        19  has_date  content_sem

In [12]:
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)

In [13]:
# 5. 效果评估
print("准确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

准确率为:0.97996
召回率为:0.98725
F1值为:0.98359
